In [ ]:
import tensorflow as tf

In [ ]:
string  = tf.Variable("This is a string", tf.string)## UpperCas 'V' for the variable
number = tf.Variable(324, tf.int16)
floating = tf.Variable(3.567, tf.float64)
#all of this are tensors of shape 0 i.e a numpy array of shape 0 aka a scalar


rank1_tensor = tf.Variable(['text'], tf.string)
rank2_tensor = tf.Variable([['text'],['text2']], tf.string)
## just as their names imply the ranks are respectively...
## to determine the rank we canjust write:
tf.rank(rank2_tensor)
#Output: <tf.Tensor: shape=(), dtype=int32, numpy=2> 

# we can also get the shape of a tensor.
rank2_tensor.shape
# Output: TensorShape([2,1]) the first value represents the number of columns and the second represents the number of elements per column.

#----- Changing Shape -----#
tensor1 = tf.ones([1,2,3])
tensor2 = tf.reshape(tensor1, [2,3,1])
tensor3 = tf.reshape(tensor2, [3,-1])# by using -1 we allow tensor to calculate the suitable shape for us.

#-----Types of tensor-----#
# Variable, Constant, Placeholder, SparseTensor

#-----Evaluating Tensors-----#
# with tf.Session() as sess:
#     tensor.eval()

t = tf.zeros([5,5,5,5])
t1 = tf.reshape(t, [25, -1])
t1

In [ ]:
#-----Core learning  Algorithms-----#
#Linear Regression
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib 
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

dftrain = pd.read_csv('train.csv')
dfeval = pd.read_csv('eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique() ## gets all unique values in the column
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype = tf.float32))
    
    
##Now that we have our data ready i.e we've loaded the data and converted categorical data into numerical, we mive to the training process.
##-----The training process-----##


## we load the data in batches unto our ram.
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size=32):
    def input_function(): # inner funcy=tion, will be returned
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
        if shuffle:
            ds = ds.shuffle(1000)# randomize order of data
        ds = ds.batch(batch_size).repeat(num_epochs)#split dataset into patches and repeat for the number of epochs
        return ds# batch the dataset
    return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs = 1, shuffle = False)

#---- creating the model-----# 
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

#----Training the model-----#
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)
clear_output()
print(result['accuracy'])

In [ ]:
result = list(linear_est.predict(eval_input_fn))# get the predictions for every entry
print(dfeval.loc[264])## get the details of the passenger
#print(y_eval.loc[-1])## survived or not
result[263]## percentage chance if survival

In [ ]:
# Import necessary libraries and modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Load the training and evaluation datasets from CSV files
dftrain = pd.read_csv('Real estate.csv')
dfeval = pd.read_csv('Realestate_eval.csv')

# Separate the target variable from the feature variables
y_train = dftrain.pop('house price of unit area')
y_eval = dfeval.pop('house price of unit area')

# Define a list of numeric feature columns for the model to use
feature_columns = [tf.feature_column.numeric_column(feature_name, dtype=tf.float32)
                   for feature_name in dftrain.columns]

# Define a function to create a `tf.data.Dataset` object for training or evaluation
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

# Create input functions for training and evaluation datasets
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

# Create a linear regression model using the TensorFlow Estimator API
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# Train the model using the training dataset
linear_est.train(train_input_fn)

# Evaluate the model using the evaluation dataset and print the accuracy score
result = linear_est.evaluate(eval_input_fn)
print(result)


In [2]:
##----- Classification------#
from IPython.display import clear_output
# Import necessary libraries and modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
##----- loading our data -----##
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header = 0)

##----- Separating our target from features ----#
train_y = train.pop('Species')
test_y = test.pop('Species')

##--- Create feature columns ---##
my_feature_columns = [tf.feature_column.numeric_column(key=key) for key in train.keys()]

##---- Input Function ----#
def input_fn(features, labels, training = True, batch_size = 256):
    #Convert the input to a dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    #Shuffle and repeaat if you are in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

##----- Building the Model -----#
## Using the DNNCLASSIFIER
# build a DNN(deep neural network) with 2 hidden layers and with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    #two hidden layers of 30 and 10 nodes respectively.
    hidden_units = [30,10],
    #the model must choose between 3 classes.
    n_classes = 3)

##----- Training the model -----##
classifier.train(
input_fn = lambda: input_fn(train,  train_y, training = True), steps = 5000)

##-----Evaluating the model ----##

classifier.evaluate(
input_fn = lambda: input_fn(test,  test_y, training = False))

clear_output()
##----- Predicting from input ----##
def predict_input_fn(features, batch_size = 256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict= {}
print("Please enter the respective numeric values")
for feature in features:
    valid = True
    while valid:
        val = input(feature + ': ')
        if not val.isdigit(): valid = False
    predict[feature] = [float(val)]
    
predictions = classifier.predict(input_fn = lambda:predict_input_fn(predict))
for pred_dict in predictions
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('prediction is "{}", ({:.1f}%)'.format(SPECIES[class_id], 100*probability))

Please enter the respective numeric values
SepalLength: 2.4
SepalWidth: 2.6
PetalLength: 6.5
PetalWidth: 6.3
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\OREO\AppData\Local\Temp\tmp3330ks9y\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
prediction is "Virginica", (56.5%)


In [36]:
##-----Clustering-----##
##----Hidden Marcov Model-----##
#%tensorflow_version 2.x
import tensorflow_probability as tfp
import tensorflow as tf
##the probability distribution were working with is a weather prediction
##the probability has already being defined as:
#1. Cold days are encoded by 0 and hot by 1
#2.The first day in our sequence has an 80% chance of being cold
#3.A cold day has a 30% chance of being followed by a hot day
#4.A hot day has a 20% chance of being followed by a cold day
#5.On each day the temperature is normally distributed with mean and standard deviation 0 and 5 on a cold day and 
#mean and standard deviation 15 and 10 on a hot day.
tfd = tfp.distributions
initial_distribution = tfd.Categorical(probs=[0.8, 0.2])
transition_distribution = tfd.Categorical(probs = [[0.7,0.3],[0.2,0.8]])
observation_distribution = tfd.Normal(loc = [0., 15.], scale = [5., 10.])
# loc: mean and scale:standard deviation
# the Model: 
model = tfd.HiddenMarkovModel(
        initial_distribution=initial_distribution,
        transition_distribution=transition_distribution,
        observation_distribution=observation_distribution,
        num_steps = 7)
mean = model.mean()
with tf.compat.v1.Session() as sess:# we need to do this to run a session.
    print(mean.numpy())

[3.        5.9999995 7.4999995 8.25      8.625     8.812501  8.90625  ]
